In [1]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import requests
from IPython import embed

In [2]:
base_url = "https://www.finn.no"
index_url = f"{base_url}/realestate/homes/search.html?location=0.20061&sort=PUBLISHED_DESC"
#page = requests.get(index_url)
#soup = BeautifulSoup(page.content, 'html.parser')

def get_soup(url):
    page = requests.get(url)
    return BeautifulSoup(page.content, 'html.parser')

soup = get_soup(index_url)
with open("index.html", "w") as f:
    f.write(soup.__str__())

In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="no">
 <head>
  <link href="https://www.finn.no/favicon.ico" rel="icon"/>
  <link href="https://www.finn.no/favicon-t-32x32.png" rel="icon" sizes="32x32"/>
  <link href="https://www.finn.no/favicon-t-192x192.png" rel="icon" sizes="192x192"/>
  <link href="https://www.finn.no/favicon-t-96x96.png" rel="icon" sizes="96x96"/>
  <link href="https://www.finn.no/favicon-t-16x16.png" rel="icon" sizes="16x16"/>
  <link href="https://www.finn.no/apple-touch-icon-180x180.png" rel="apple-touch-icon"/>
  <meta content="#06befb" name="msapplication-TileColor"/>
  <meta content="https://www.finn.no/mstile-144x144.png" name="msapplication-TileImage"/>
  <meta content="#06befb" name="theme-color"/>
  <meta content="https://www.finn.no/browserconfig.xml" name="msapplication-config"/>
  <link href="https://www.finn.no/manifest.json" rel="manifest"/>
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
        new Date().getTime(),event:'gtm.js'});var f=d.get

In [6]:
card = cards[0]
card.h2.a["href"]

'/realestate/homes/ad.html?finnkode=201988281'

In [7]:
def extract_int(s) -> int:
    return int("".join(digit for digit in s if digit.isdigit()))

# url_from_Cards
card.find_all("a")[0]["href"]
relative_urls = {card.h2.get_text(): card.h2.a["href"] for card in cards}

In [10]:
# find all pages

def find_all_pages(soup, start_side: int, base_url):
    pages = []
    current_page = start_side
    for page in soup.find_all("a", {"pagination__page button button--pill"}):
        curent_page = extract_int(page["aria-label"])
        if curent_page > start_side:
            pages.append(page)
    if len(pages) == 0:
        return []
    new_soup = get_soup(f"{base_url}{pages[-1]['href']}" )
    # embed(header="just created new soup")
    return pages + find_all_pages(
        new_soup, curent_page, base_url)
    
pages = find_all_pages(soup, start_side=0, base_url="https://www.finn.no/realestate/homes/search.html")
pages

[<a aria-current="page" aria-label="Side 1" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC">1</a>,
 <a aria-label="Side 2" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=2">2</a>,
 <a aria-label="Side 3" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=3">3</a>,
 <a aria-label="Side 4" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=4">4</a>,
 <a aria-label="Side 5" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=5">5</a>,
 <a aria-label="Side 6" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=6">6</a>,
 <a aria-label="Side 7" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=7">7</a>,
 <a aria-label="Side 8" class="paginati

In [57]:
# extract data from listing
kw = "definition-list definition-list--cols1to2"
url = "https://www.finn.no/realestate/homes/ad.html?finnkode=203500388"
soup = get_soup(url)
dl_table = soup.find("dl", {"class": kw})
data = {dt.get_text(): dt.find_next("dd").get_text() for dt in dl_table.find_all("dt")}
dts = dl_table.find_all("dt")
dds = [dt.find_next("dd") for dt in dts]

In [58]:
data

{'Boligtype': 'Leilighet',
 'Eieform bolig': 'Andel',
 'Soverom': '2',
 'Primærrom': '67 m²',
 'Bruksareal': '67 m²',
 'Etasje': '4',
 'Byggeår': '1956',
 'Energimerking': '\n\n                            G - rød\n\n'}

In [40]:
itr = dl_table.__iter__()


In [41]:
dt = next(itr)

In [48]:
print(dt)b

KeyError: 2

## New session


In [24]:
from typing import List
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import requests

import util
from IPython import embed

In [25]:
base_url = "https://www.finn.no"
search_url = f"{base_url}/realestate/homes/search.html"
index_url = f"{search_url}?location=0.20061&sort=PUBLISHED_DESC"

In [38]:
def get_soup(url):
    page = requests.get(url)
    return BeautifulSoup(page.content, 'html.parser')


def get_card_links(soup):
    cards = {card.h2.a["href"] for card in soup.find_all(
        "article", {"class": "ads__unit"})}
    return cards


def get_all_page_links(soup, start_side: int, base_url) -> List[str]:
    """
    Getting links to all pages in the index

    Args:
        soup: bf4.BeautifulSoup
            index soup
        start_site: int
            Side to incude from
        base url: str
            base url for adding to href
    Return: List[str]
        list of hrefs to pages
    """
    pages = []
    current_page = start_side
    for page in soup.find_all("a", {"pagination__page button button--pill"}):
        current_page = util.extract_int(page["aria-label"])
        if current_page > start_side:
            pages.append(page["href"])
    if len(pages) == 0:
        return []
    new_soup = get_soup(f"{base_url}{pages[-1]}")
    return pages + get_all_page_links(
        new_soup, current_page, base_url)


def get_data(soup):
    kw = "definition-list definition-list--cols1to2"
    dl_table = soup.find("dl", {"class": kw})
    return {dt.get_text(): dt.find_next("dd").get_text()
            for dt in dl_table.find_all("dt")}

In [27]:
index_soup = get_soup(index_url)

In [30]:
# get all pages
page_links = get_all_page_links(
    index_soup, start_side=0, base_url="https://www.finn.no/realestate/homes/search.html")

In [32]:
page_links[0]

'?location=0.20061&sort=PUBLISHED_DESC'

In [39]:
# get all cards
card_links = []
for page_link in page_links:
    for card_link in get_card_links(get_soup(search_url + page_link)):
        if "http" in card_link:
            card_links.append(card_link)
        # making all links absolute,because not all was.
        else:
            card_links.append("https://www.finn.no" + card_link)

In [40]:
for card in card_links:
    print(card)

https://www.finn.no/realestate/homes/ad.html?finnkode=204344379
https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=179301937&location=0.20061&sort=PUBLISHED_DESC
https://www.finn.no/realestate/homes/ad.html?finnkode=204332785
https://www.finn.no/realestate/homes/ad.html?finnkode=204438115
https://www.finn.no/realestate/homes/ad.html?finnkode=204364662
https://www.finn.no/realestate/homes/ad.html?finnkode=204318380
https://www.finn.no/realestate/homes/ad.html?finnkode=203825204
https://www.finn.no/realestate/homes/ad.html?finnkode=204464876
https://www.finn.no/realestate/homes/ad.html?finnkode=191600090
https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=204452409&location=0.20061&sort=PUBLISHED_DESC
https://www.finn.no/realestate/homes/ad.html?finnkode=204176790
https://www.finn.no/realestate/homes/ad.html?finnkode=173010432
https://www.finn.no/realestate/homes/ad.html?finnkode=203076097
https://www.finn.no/realestate/homes/ad.html?finnkode=204380681
https://www.finn.no/realestate/h

In [45]:
# get data from cards
card_data = []
N = len(card_links)
N_failed = 0
for i, card_link in enumerate(card_links):
    try:
        card_data.append(
            {"url": card_link, "data": get_data(get_soup(card_link))}
        )
    except AttributeError as e:
        print(f"url: {card_link}\n{e}")
        print(e)
        N_failed += 1
    #p = (i/N)*100
    print(f"{(i/N)*100}. {i} out of {N}, {N_failed=}")
        

0.0. 0 out of 1201, N_failed=0
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=179301937&location=0.20061&sort=PUBLISHED_DESC
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
0.08326394671107411. 1 out of 1201, N_failed=1
0.16652789342214822. 2 out of 1201, N_failed=1
0.2497918401332223. 3 out of 1201, N_failed=1
0.33305578684429643. 4 out of 1201, N_failed=1
0.4163197335553705. 5 out of 1201, N_failed=1
0.4995836802664446. 6 out of 1201, N_failed=1
0.5828476269775187. 7 out of 1201, N_failed=1
0.6661115736885929. 8 out of 1201, N_failed=1
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=204452409&location=0.20061&sort=PUBLISHED_DESC
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
0.7493755203996669. 9 out of 1201, N_failed=2
0.832639467110741. 10 out of 1201, N_failed=2
0.9159034138218152. 11 out of 1201, N_failed=2
0.9991673605328892. 12 out of 1201, N_failed=2
1.0824313072439635

11.490424646128226. 138 out of 1201, N_failed=9
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=200636262&location=0.20061&sort=PUBLISHED_DESC&page=3
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
11.5736885928393. 139 out of 1201, N_failed=10
11.656952539550375. 140 out of 1201, N_failed=10
11.740216486261449. 141 out of 1201, N_failed=10
11.823480432972524. 142 out of 1201, N_failed=10
11.906744379683596. 143 out of 1201, N_failed=10
11.99000832639467. 144 out of 1201, N_failed=10
12.073272273105745. 145 out of 1201, N_failed=10
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=201226477&location=0.20061&sort=PUBLISHED_DESC&page=3
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
12.156536219816818. 146 out of 1201, N_failed=11
12.239800166527894. 147 out of 1201, N_failed=11
12.323064113238967. 148 out of 1201, N_failed=11
12.406328059950042. 149 out of 1201, N_failed=11
12.48959

21.73189009159034. 261 out of 1201, N_failed=20
21.815154038301415. 262 out of 1201, N_failed=20
21.89841798501249. 263 out of 1201, N_failed=20
21.981681931723564. 264 out of 1201, N_failed=20
22.06494587843464. 265 out of 1201, N_failed=20
22.148209825145713. 266 out of 1201, N_failed=20
22.231473771856784. 267 out of 1201, N_failed=20
22.31473771856786. 268 out of 1201, N_failed=20
22.398001665278937. 269 out of 1201, N_failed=20
22.481265611990008. 270 out of 1201, N_failed=20
22.564529558701082. 271 out of 1201, N_failed=20
22.647793505412157. 272 out of 1201, N_failed=20
22.73105745212323. 273 out of 1201, N_failed=20
22.814321398834306. 274 out of 1201, N_failed=20
22.89758534554538. 275 out of 1201, N_failed=20
22.98084929225645. 276 out of 1201, N_failed=20
23.064113238967526. 277 out of 1201, N_failed=20
23.1473771856786. 278 out of 1201, N_failed=20
23.230641132389675. 279 out of 1201, N_failed=20
23.31390507910075. 280 out of 1201, N_failed=20
23.397169025811824. 281 out of

34.47127393838468. 414 out of 1201, N_failed=24
34.554537885095755. 415 out of 1201, N_failed=24
34.63780183180683. 416 out of 1201, N_failed=24
34.721065778517904. 417 out of 1201, N_failed=24
34.804329725228975. 418 out of 1201, N_failed=24
34.887593671940046. 419 out of 1201, N_failed=24
34.970857618651124. 420 out of 1201, N_failed=24
35.0541215653622. 421 out of 1201, N_failed=24
35.13738551207327. 422 out of 1201, N_failed=24
35.22064945878435. 423 out of 1201, N_failed=24
35.30391340549542. 424 out of 1201, N_failed=24
35.38717735220649. 425 out of 1201, N_failed=24
35.47044129891757. 426 out of 1201, N_failed=24
35.55370524562864. 427 out of 1201, N_failed=24
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=203608453&location=0.20061&sort=PUBLISHED_DESC&page=9
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
35.63696919233972. 428 out of 1201, N_failed=25
35.72023313905079. 429 out of 1201, N_failed=25
35.80349708576186. 430 o

url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=187567919&location=0.20061&sort=PUBLISHED_DESC&page=11
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
45.21232306411324. 543 out of 1201, N_failed=37
45.29558701082431. 544 out of 1201, N_failed=37
45.37885095753539. 545 out of 1201, N_failed=37
45.46211490424646. 546 out of 1201, N_failed=37
45.54537885095753. 547 out of 1201, N_failed=37
45.62864279766861. 548 out of 1201, N_failed=37
45.71190674437968. 549 out of 1201, N_failed=37
45.79517069109076. 550 out of 1201, N_failed=37
45.87843463780183. 551 out of 1201, N_failed=37
45.9616985845129. 552 out of 1201, N_failed=37
46.04496253122398. 553 out of 1201, N_failed=37
46.12822647793505. 554 out of 1201, N_failed=37
46.21149042464613. 555 out of 1201, N_failed=37
46.2947543713572. 556 out of 1201, N_failed=37
46.37801831806828. 557 out of 1201, N_failed=37
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=203566260&location=

...

75.77019150707743. 910 out of 1201, N_failed=71
75.85345545378851. 911 out of 1201, N_failed=71
75.93671940049958. 912 out of 1201, N_failed=71
76.01998334721067. 913 out of 1201, N_failed=71
76.10324729392173. 914 out of 1201, N_failed=71
76.1865112406328. 915 out of 1201, N_failed=71
76.26977518734388. 916 out of 1201, N_failed=71
76.35303913405495. 917 out of 1201, N_failed=71
76.43630308076604. 918 out of 1201, N_failed=71
76.5195670274771. 919 out of 1201, N_failed=71
76.60283097418818. 920 out of 1201, N_failed=71
76.68609492089925. 921 out of 1201, N_failed=71
76.76935886761032. 922 out of 1201, N_failed=71
76.8526228143214. 923 out of 1201, N_failed=71
76.93588676103246. 924 out of 1201, N_failed=71
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=173104120&location=0.20061&sort=PUBLISHED_DESC&page=19
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
77.01915070774355. 925 out of 1201, N_failed=72
url: https://www.finn.no/eiend

81.09908409658618. 974 out of 1201, N_failed=97
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=156178662&location=0.20061&sort=PUBLISHED_DESC&page=20
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
81.18234804329725. 975 out of 1201, N_failed=98
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=201435477&location=0.20061&sort=PUBLISHED_DESC&page=20
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
81.26561199000832. 976 out of 1201, N_failed=99
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=201895148&location=0.20061&sort=PUBLISHED_DESC&page=20
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
81.3488759367194. 977 out of 1201, N_failed=100
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=143926786&location=0.20061&sort=PUBLISHED_DESC&page=20
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute

url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=141466534&location=0.20061&sort=PUBLISHED_DESC&page=20
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
84.59616985845129. 1016 out of 1201, N_failed=130
84.67943380516236. 1017 out of 1201, N_failed=130
84.76269775187345. 1018 out of 1201, N_failed=130
84.84596169858452. 1019 out of 1201, N_failed=130
84.92922564529559. 1020 out of 1201, N_failed=130
85.01248959200666. 1021 out of 1201, N_failed=130
85.09575353871773. 1022 out of 1201, N_failed=130
85.17901748542882. 1023 out of 1201, N_failed=130
85.26228143213989. 1024 out of 1201, N_failed=130
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=199069438&location=0.20061&sort=PUBLISHED_DESC&page=21
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
85.34554537885096. 1025 out of 1201, N_failed=131
85.42880932556203. 1026 out of 1201, N_failed=131
url: https://www.finn.no/eiendom/nybygg/

89.50874271440466. 1075 out of 1201, N_failed=155
89.59200666111575. 1076 out of 1201, N_failed=155
89.6752706078268. 1077 out of 1201, N_failed=155
89.75853455453789. 1078 out of 1201, N_failed=155
89.84179850124896. 1079 out of 1201, N_failed=155
89.92506244796003. 1080 out of 1201, N_failed=155
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=193549148&location=0.20061&sort=PUBLISHED_DESC&page=22
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
90.00832639467112. 1081 out of 1201, N_failed=156
90.09159034138217. 1082 out of 1201, N_failed=156
90.17485428809326. 1083 out of 1201, N_failed=156
90.25811823480433. 1084 out of 1201, N_failed=156
90.3413821815154. 1085 out of 1201, N_failed=156
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=193989369&location=0.20061&sort=PUBLISHED_DESC&page=22
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
90.42464612822648. 1086 out of 1201, N_fai

95.00416319733556. 1141 out of 1201, N_failed=179
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=190183577&location=0.20061&sort=PUBLISHED_DESC&page=23
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
95.08742714404663. 1142 out of 1201, N_failed=180
95.1706910907577. 1143 out of 1201, N_failed=180
95.25395503746877. 1144 out of 1201, N_failed=180
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=192200374&location=0.20061&sort=PUBLISHED_DESC&page=23
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
95.33721898417986. 1145 out of 1201, N_failed=181
95.42048293089093. 1146 out of 1201, N_failed=181
95.503746877602. 1147 out of 1201, N_failed=181
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=190309323&location=0.20061&sort=PUBLISHED_DESC&page=23
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
95.58701082431307. 1148 out of 1201, N

In [47]:
card_data[2]


{'https://www.finn.no/realestate/homes/ad.html?finnkode=204438115': 'https://www.finn.no/realestate/homes/ad.html?finnkode=204438115',
 'data': {'Boligtype': 'Leilighet',
  'Eieform bolig': 'Eier (Selveier)',
  'Soverom': '1',
  'Primærrom': '76 m²',
  'Bruksareal': '76 m²',
  'Etasje': '1',
  'Byggeår': '1983',
  'Energimerking': '\n\n                            F - mørkegrønn\n\n'}}

In [68]:
soup = get_soup("https://www.finn.no/realestate/homes/ad.html?finnkode=204438115").find("main", {"class": "pageholder"})
data = {
    "url": "https://www.finn.no/realestate/homes/ad.html?finnkode=204438115",
    "header": "",
    "about": None,
}


In [73]:
header = soup.find("h1")
data["header"] = header.get_text()
data["header"]

'SKØYEN TERRASSE- Stor 2 - roms med peis, usjenert 17 kvm solrik sydvestvendt terrasse, garasje, heis, rolig og sentralt'

In [84]:
location

<p class="u-caption">Skøyen Terrasse 32, 0276 Oslo</p>

In [105]:
panel = location.find_next("div", {"class":"panel"})
price = panel.find_all("span")[1].get_text()
price

'\n        6\xa0000\xa0000 kr'

In [93]:
print(data["price"])
print(panel)



<div class="panel">
<span class="u-strong u-display-block">Prisantydning</span>
<span class="u-t3">
        6 000 000 kr</span>
<dl class="definition-list">
</dl>
<dl class="definition-list">
<dt>Fellesgjeld</dt>
<dd>216 178 kr</dd>
<dt>Omkostninger</dt>
<dd>165 675 kr</dd>
<dt>Totalpris</dt>
<dd>6 381 853 kr</dd>
<dt>Felleskost/mnd.</dt>
<dd>4 748 kr</dd>
</dl>
</div>


In [ ]:
# type of realestate
kw = "definition-list definition-list--cols1to2"
about_table = soup.find("dl", {"class": kw})
data["about"] = {dt.get_text(): dt.find_next("dd").get_text()
                 for dt in about_table.find_all("dt")}